In [0]:
spark.sql("USE CATALOG hp_az_catalog")
spark.sql("USE SCHEMA hp_iris_ml")

In [0]:
dbutils.widgets.text("pkl_path", "/Volumes/hp_az_catalog/hp_iris_ml/hp_ml_vol/iris_model.pkl")
dbutils.widgets.text("experiment_path", "/Users/hemapriya.nagarajan@databricks.com/models/hp_iris_ml")

In [0]:
import pandas as pd
from sklearn.datasets import load_iris
import pickle
import mlflow
from mlflow.models.signature import infer_signature

pkl_path = dbutils.widgets.get("pkl_path")
experiment_path = dbutils.widgets.get("experiment_path")
model_name = 'iris_model'
with open(pkl_path, "rb") as f:
    model = pickle.load(f)

# Sample data to infer schema
iris = load_iris()
sample = pd.DataFrame(iris.data, columns=iris.feature_names)
preds = model.predict(sample)
signature = infer_signature(sample, preds)

class SklearnWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import pickle
        with open(context.artifacts["model_path"], "rb") as f:
            self.model = pickle.load(f)
    def predict(self, context, model_input):
        return self.model.predict(model_input)

In [ ]:
import mlflow
exp = mlflow.get_experiment_by_name("/Users/hemapriya.nagarajan@databricks.com/models/hp_iris_ml")
print(exp)

In [0]:
import mlflow
# mlflow.set_experiment(exp.experiment_id)

with mlflow.start_run(experiment_id=exp.experiment_id):
    artifacts = {"model_path": pkl_path}
    mlflow.pyfunc.log_model(
        artifact_path="pickle_model",
        python_model=SklearnWrapper(),
        artifacts=artifacts,
        signature=signature,
    )
    run_id = mlflow.active_run().info.run_id

# Register to Model Registry
model_uri = f"runs:/{run_id}/pickle_model"
model_details = mlflow.register_model(model_uri, "IrisClassifierDemo")
print("Registered:", model_details)

In [0]:
# Out of all runs in the experiment, only register the run with the best selected metric
# If you want, you can simply register the latest run instead, for example.
client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_path)
print(experiment)

In [0]:
# runs = client.search_runs(experiment_ids=[experiment.experiment_id], order_by=[f"metrics.{selected_metric} DESC"], max_results=1)
runs = client.search_runs(experiment_ids=[exp.experiment_id], max_results=1)
best_run_id = runs[0].info.run_id
# print(best_run_id)
model_uri = f"runs:/{best_run_id}/pickle_model"
registered_model = mlflow.register_model(model_uri, "IrisClassifierDemo")
client.set_registered_model_alias(name=registered_model.name, alias="challenger", version=registered_model.version)
print(registered_model)